In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from torch.optim import AdamW
pd.options.mode.chained_assignment = None
import warnings
import random

Matplotlib is building the font cache; this may take a moment.


In [3]:
import torch
import torch.nn as nn
from model import ManZoneTransformer

In [8]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

model = ManZoneTransformer(
    feature_len=5,    # num of input features (x, y, v_x, v_y, defense)
    model_dim=64,     # experimented with 96 & 128... seems best
    num_heads=2,      # 2 seems best (but may have overfit when tried 4... may be worth iterating)
    num_layers=4,
    dim_feedforward=64 * 4,
    dropout=0.1,      # 10% dropout to prevent overfitting... iterate as model becomes more complex (industry std is higher, i believe)
    output_dim=2      # man or zone classification
).to(device)

In [10]:
best_model_path = f"best_model_week3.pth"
model.load_state_dict(torch.load(
    best_model_path, 
    weights_only=True,
    map_location=torch.device('cpu') 
    ))
model.eval()

ManZoneTransformer(
  (feature_norm_layer): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (feature_embedding_layer): Sequential(
    (0): Linear(in_features=5, out_features=64, bias=True)
    (1): ReLU()
    (2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (3): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (drop